# <b>Start here</b> - Initial environment setup
Ensure to run each step in order - Optional steps may be skipped if desired

In [ ]:
#@title Check which Nvidia GPU has been assigned to you
!nvidia-smi

In [ ]:
#@title Step 1 - Set up the environment (Mandatory)
onGoogleColab = True
onWindows = False
previouslySetup = False
GPUID = 0

import os

ffmpeg4 = 'ffmpeg'
nvencPresetLocal = 'p7'
if onGoogleColab:
    !wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
    !tar -xf ffmpeg-release-amd64-static.tar.xz
    !rm ffmpeg-release-amd64-static.tar.xz

    ffmpegdir = 'ffmpeg-4.3.1-amd64-static'
    dirContent = os.listdir()
    for objName in dirContent:
        if 'ffmpeg' in objName:
            ffmpegdir = objName
            print('Found ffmpeg dir:',ffmpegdir)
    ffmpeg4 = '/content/'+ffmpegdir+'/ffmpeg'
    nvencPresetLocal = 'slow'

print('NVENC PRESET',nvencPresetLocal)
!git clone --branch main --recurse-submodules https://github.com/HeylonNHP/RIFE-Colab.git RIFEcolab

os.chdir('RIFEcolab/')

import sys

# Why should we need to add the submodule to the path, just for the RIFE import to work
# Thanks for being consistently terrible, python
sys.path.insert(0, os.getcwd() + os.path.sep + 'arXiv2020RIFE')

from generalInterpolationProceedures import *
setGPUinterpolationOptions(3,[0])
setNvencSettings(GPUID,nvencPresetLocal)
setFFmpeg4Path(ffmpeg4)
print('NVENC PRESET',nvencPresetLocal)

In [ ]:
#@title Step 2 - Mount your google drive to use as storage for input/output videos (Optional)
from google.colab import drive
drive.mount('/content/drive')

# Processing a single video (Extract -> Interpolate -> Create output - All at once)
**Processing a single video has now been broken into two steps:**<br>
Run the **set settings for interpolation** step first to apply the desired settings for interpolation, then run the second step to start the interpolator. <br><br>
Paste in the path to your video, choose your desired settings, then run this code box to interpolate a single video
<br>
The output will be in the same folder as the input
<br><br>
<<< - Click the folder icon on the left hand side to open the file browser<br>
Navigate to the file you'd like to interpolate<br>
Right click it, click Copy Path and paste it over the top of the example path below

In [ ]:
#@title Run to set settings for interpolation (DO FIRST)
#@markdown Input video path
inputFile = 'input.mp4' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode =  3#@param{type:"number"}
#@markdown Enable perfect loop (For GIFs or looping videos)
loopable = True #@param{type:"boolean"}
#@markdown Interpolation factor (2, 4, 8, 16, 32... etc)
interpolationFactor = 16 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = False #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.20 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}
#@markdown Use NVENC for output
useNvenc = True #@param{type:"boolean"}
#@markdown Use auto encode
autoEncode = True #@param{type:"boolean"}
#@markdown Auto encode block size
autoBlockSize = 3000 #@param{type:"number"}

In [ ]:
#@title Interpolate a single video - Run interpolator (DO SECOND)
performAllSteps(inputFile, interpolationFactor, loopable, mode, crf, clearPNGs, nonLocalPNGs, scenechangeSensitivity, mpdecimateSensitivity, useNvenc, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,useAccurateFPS=False,accountForDuplicateFrames=True)

# Processing a single video (Run steps individually)
Please run **Run to set settings for interpolation (DO FIRST)** first, before running the steps below in order **Extract frames -> Run interpolator -> Create output**
<br><br>
Running the steps individually like this will allow you to extract the frames, check for duplicate frames that were missed by FFmpeg after extraction and manually delete them, then run the interpolator, and check for any missed scene changes in the interpolated frames before creating the output

In [ ]:
#@title Extract frames (step 1)
performAllSteps(inputFile, interpolationFactor, loopable, mode, crf, clearPNGs, nonLocalPNGs, scenechangeSensitivity, mpdecimateSensitivity, useNvenc, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,useAccurateFPS=False,accountForDuplicateFrames=True,step1=True,step2=False,step3=False)

In [ ]:
#@title Run interpolator (step 2)
performAllSteps(inputFile, interpolationFactor, loopable, mode, crf, clearPNGs, nonLocalPNGs, scenechangeSensitivity, mpdecimateSensitivity, useNvenc, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,useAccurateFPS=False,accountForDuplicateFrames=True,step1=False,step2=True,step3=False)

In [ ]:
#@title Create output (step 3)
performAllSteps(inputFile, interpolationFactor, loopable, mode, crf, clearPNGs, nonLocalPNGs, scenechangeSensitivity, mpdecimateSensitivity, useNvenc, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,useAccurateFPS=False,accountForDuplicateFrames=True,step1=False,step2=False,step3=True)

# Processing multiple videos using the batch interpolator
set inputDirectory to equal the folder path containing the input videos
<br><br>
Each video must be in its own subdirectory within the input folder
<br><br>
The interpolation factor for each video is chosen based on its input FPS (Accounting for duplicate frames if using mode 4). It will choose an interpolation factor which will meet or exceed the 'Output FPS target'
<br><br>
For perfectly looping GIFs/videos: Add [l] to the filename or subfolder name of the input file. E.G.:<br>
inputLoopingGifFileNumberOne[l].gif
<br>
or
<br>
InputLoopingGifFolder[l]/inputLoopingGifFileNumberOne.gif

In [ ]:
#@title Interpolate a folder of videos
#@markdown Input folder path
inputDirectory = '/content/drive/' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode = 3 #@param{type:"number"}
#@markdown Output FPS target
fpsTarget = 59 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = True #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.20 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}
#@markdown Use NVENC for output
useNvenc = True #@param{type:"boolean"}
#@markdown Use auto encode
autoEncode = True #@param{type:"boolean"}
#@markdown Auto encode block size
autoBlockSize = 3000 #@param{type:"number"}

batchInterpolateFolder(inputDirectory,mode,crf,fpsTarget,clearPNGs,nonLocalPNGs,
                           scenechangeSensitivity,mpdecimateSensitivity,useNvenc,useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,accountForDuplicateFrames=True,useAccurateFPS=False)